# A Premier on PyTheory

PyTheory is a music theory library, which procedurally generates all known Western tones, scales, and chord fingering charts (for custom fretboards).

It is capable of outputting either a rounded decimal for pitch representation, or the *proper* symbolic representation of the pitch (as a SymPy object).


## Install PyTheory:

In [1]:
!pip install pytheory requests

## Import PyTheory:

In [2]:
from pytheory import TonedScale

Let's reference a great key (and the only one I can play on piano), A Minor:


In [3]:
a_min = TonedScale(tonic="G#4")["minor"]
a_min

<Scale I=G#4 II=A#5 III=B5 IV=C#5 V=D#5 VI=E5 VII=F#5 VIII=G#5>

Let's reference another great key, C Major:

In [4]:
c_maj = TonedScale(tonic="C4")["major"]
c_maj

<Scale I=C4 II=D4 III=E4 IV=F4 V=G4 VI=A5 VII=B5 VIII=C5>

That's interesting, they contain all the same notes! 

----------------

Let's see if we can automatically detect what the modal correspondents are to A Minor:

In [5]:
from pytheory import SYSTEMS

def get_scale(tonic, mode):
  return TonedScale(tonic=tonic)[mode]

def get_all_scales(octave=4):  
  scales = []
  
  # Iterate over all 12 tones of the Western system:  
  for tone in SYSTEMS["western"].tones:
    
    # Create a scale for each tone.
    new_scale = TonedScale(tonic=f"{tone.name}{octave}")
         
    for mode in new_scale._scales.keys():
      # Create every possible permutation of the scale.      
      newest_scale = TonedScale(tonic=f"{tone.name}{octave}")[mode]
      
      # (tone, mode, scale) tuple.       
      scales.append((tone, mode, newest_scale))
      
  return scales


def get_matches_filter(scale):
  
  def get_good_tones(scale):
    good_tones = list(scale.tones)

    # Add an upper and lower octave to the list of good known tones.   
    for i in range(len(SYSTEMS["western"].tones)):
      good_tones.append(good_tones[i].add(12))
      good_tones.append(good_tones[i].add(-12))
      
    return good_tones

  def matches_the_key(tmst):
    # Expand the (tone, mode, scale) tuple.   
    tone, mode, other_scale = tmst

    trusted = True
    good_tones = get_good_tones(scale)
    
    # Iterate over each tone:  
    for tone in other_scale.tones:

      # If the tone isn't in a set of known good tones, don't trust it.
      if tone not in good_tones:
        trusted = False

    return trusted
  
  return matches_the_key

In [0]:
# Here's the good stuff:
scale = get_scale(tonic="G#4", mode="minor")

print(f"Scales that match A minor:\n")

for tmst in filter(get_matches_filter(scale), get_all_scales()):
  tone, mode, other_scale = tmst
  
  # Filter out chromatic scales.   
  if len(other_scale.tones) == len(scale.tones):
    print(f"{tone.name} {mode}: {other_scale}")

Scales that match A minor:

A# ionian: <Scale I=A#4 II=B4 III=C#4 IV=D#4 V=E4 VI=F#4 VII=G#4 VIII=A#5>
A# aeolian: <Scale I=A#4 II=B4 III=C#4 IV=D#4 V=E4 VI=F#4 VII=G#4 VIII=A#5>
B major: <Scale I=B4 II=C#4 III=D#4 IV=E4 V=F#4 VI=G#4 VII=A#5 VIII=B5>
D# dorian: <Scale I=D#4 II=E4 III=F#4 IV=G#4 V=A#5 VI=B5 VII=C#5 VIII=D#5>
E phrygian: <Scale I=E4 II=F#4 III=G#4 IV=A#5 V=B5 VI=C#5 VII=D#5 VIII=E5>
F# lydian: <Scale I=F#4 II=G#4 III=A#5 IV=B5 V=C#5 VI=D#5 VII=E5 VIII=F#5>
G# minor: <Scale I=G#4 II=A#5 III=B5 IV=C#5 V=D#5 VI=E5 VII=F#5 VIII=G#5>
G# locrian: <Scale I=G#4 II=A#5 III=B5 IV=C#5 V=D#5 VI=E5 VII=F#5 VIII=G#5>
G# mixolydian: <Scale I=G#4 II=A#5 III=B5 IV=C#5 V=D#5 VI=E5 VII=F#5 VIII=G#5>


## Playing With Pitch:

PyTheory also supports some advanced pitch exploration:

In [6]:
c_min = TonedScale(tonic='C4')['minor']
c_min

<Scale I=C4 II=D4 III=D#4 IV=F4 V=G4 VI=G#4 VII=A#5 VIII=C5>

In [7]:
g4 = c_min["mixolydian"]
g4

<Tone G4>

In [8]:
# Rounded decimal.
g4.pitch()

783.990871963499

In [9]:
g4.pitch(symbolic=True)

440*2**(5/6)

In [0]:
g4.pitch(temperament="pythagorean", symbolic=True)

7040/9